**Basic Exploration of the dataset

1.1. List the data type of the columns

1.2. List the names of the columns in the dataframe

1.3. Count of rows in the DF

1.4. Count of columns in the DF


1.5. Explore values in a dataframe

Select a subset of the dataframe to create a new dataframe

2.1 Select a subset of columns

2.2 Select a subset of row based on filter

2.3 Select specific rows and columns

Create a new column

Creating a pivot table

Join 2 dataframes

Convert hiredate to_date

Writing dataframe back to disk

In [0]:
df = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("/FileStore/tables/City_wide_Electricity_Mix__1_.csv")
df = df.select([F.col(col).alias(col.replace(' ', '_')) for col in df.columns])

In [0]:
#column name changing
df = df.select([F.col(col).alias(col.replace(' ', '_')) for col in df.columns])

In [0]:
#Handling Missing Values
df = df.na.drop()
df.count()
df = df.withColumn("horsepower", df["horsepower"].cast(IntegerType())) #convert horsepower from string to int
#df.display(5)
emp_df.printSchema()                   # Notice the hiredate is also string
print(emp_df.columns)                  # list headers in the DF
print(emp_df.count())                  # 14 rows
print(len(emp_df.columns))             # 8 columns
emp_df.describe('hiredate').show()     # date is taken as string hence describe does not give correct MEAN, STDEV, MIN, MAX 
emp_df.describe('sal').show()          # works best for numeric column
emp_df.describe('com').show()
emp_df.describe('ename').show()

In [0]:
df.printSchema() 

root
 |-- City_long_name: string (nullable = true)
 |-- City_short_name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Population_year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Access: string (nullable = true)
 |-- CDP_reporting_year: integer (nullable = true)
 |-- Biomass_(%): double (nullable = true)
 |-- Coal_(%): double (nullable = true)
 |-- Gas_(%): double (nullable = true)
 |-- Geothermal_(%): double (nullable = true)
 |-- Hydro_(%): double (nullable = true)
 |-- Nuclear_(%): double (nullable = true)
 |-- Oil_(%): double (nullable = true)
 |-- Solar_(%): double (nullable = true)
 |-- Wind_(%): double (nullable = true)
 |-- Unknown_sources_(%): double (nullable = true)
 |-- City_Location: string (nullable = true)



In [0]:
print(df.select('City_long_name').distinct().count())  # Count of distinct values
df.select('Country').distinct().show()          # Display those distinct values
df.select('City_long_name','Country').distinct().show() # Works on multiple columns too
df.dropDuplicates(['City_long_name','Country']).select('City_long_name','Country').show()  # Alternate method using dropDuplicates

338
+------------------+
|           Country|
+------------------+
|State of Palestine|
|            France|
|            Taiwan|
|         Argentina|
|           Ecuador|
|             India|
|           Nigeria|
|             Italy|
|             Spain|
|           Denmark|
|            Panama|
|            Israel|
|               USA|
|       South Korea|
|            Mexico|
|            Zambia|
|            Canada|
|        Mozambique|
|            Brazil|
|            Poland|
+------------------+
only showing top 20 rows

+--------------------+-----------+
|      City_long_name|    Country|
+--------------------+-----------+
|Tel Aviv-Yafo Mun...|     Israel|
|Municipalidad de ...|  Argentina|
|  Alcaldía de Panamá|     Panama|
|Prefeitura Munici...|     Brazil|
|     City of Goiânia|     Brazil|
|  City of London, ON|     Canada|
|     City of Wroclaw|     Poland|
|    Town of Ajax, ON|     Canada|
|   City of Davis, CA|        USA|
| Municipio de Mérida|     Mexico|
|Frederiksh

Filter out NULL values of a column.

In [0]:
df.dropna(subset=('Wind_(%)')).display()
df.filter(col('Wind_(%)').isNotNull()).display()     # Alternate method

City_long_name,City_short_name,Country,Region,Population_year,Population,Access,CDP_reporting_year,Biomass_(%),Coal_(%),Gas_(%),Geothermal_(%),Hydro_(%),Nuclear_(%),Oil_(%),Solar_(%),Wind_(%),Unknown_sources_(%),City_Location
Prefeitura Municipal de Caxias do Sul,Caxias do Sul,Brazil,Latin America,2016,435664,Public,2016,0.0,25.0,0.0,0.0,72.0,0.0,0.0,0.0,3.0,0.0,"(-29.163403, -51.179668)"
Incheon Metropolitan City,Incheon,South Korea,East Asia,2015,2983484,Public,2016,0.64,0.5,15.1,0.01,0.02,0.0,63.5,0.01,0.03,20.19,"(37.456256, 126.705206)"
Prefeitura Municipal da Estância Hidromineral de Águas de São Pedro,Estância Hidromineral de Águas de São Pedro,Brazil,Latin America,2014,3007,Public,2016,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,"(-22.600006, -47.875041)"
City of Wroclaw,Wroclaw,Poland,Europe,2014,634487,Public,2017,0.0,96.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,"(51.107885, 17.038538)"
Distrito Metropolitano de Quito,Quito,Ecuador,Latin America,2010,2239191,Public,2017,0.0,0.0,0.0,0.0,69.0,0.0,31.0,0.0,0.0,0.0,"(-0.2333, -78.5167)"
City of Philadelphia,Philadelphia,USA,North America,2014,1560297,Public,2017,1.28,35.27,20.62,0.0,1.0,39.9,0.55,0.01,0.67,0.7,"(39.952335, -75.163789)"
Abasan Al-Kabira Municipality,Abasan Al-Kabira,State of Palestine,South and West Asia,2015,30000,Public,2017,5.0,17.0,18.0,0.0,0.0,0.0,50.0,10.0,0.0,0.0,"(31.323126, 34.344025)"
"City of Davis, CA","Davis, CA",USA,North America,2010,65600,Public,2017,5.0,0.0,45.0,5.0,8.0,21.0,0.0,9.0,7.0,0.0,"(38.544907, -121.740517)"
"City of London, ON","London, ON",Canada,North America,2014,375000,Public,2017,0.4,0.0,9.7,0.0,23.0,58.0,0.0,1.9,6.4,0.6,"(42.9837, -81.2497)"
Frederikshavn Kommune,Frederikshavn Kommune,Denmark,Europe,2014,23156,Public,2017,14.0,25.96,23.8,0.02,0.02,0.0,0.0,0.2,36.0,0.0,"(57.442711, 10.521006)"


City_long_name,City_short_name,Country,Region,Population_year,Population,Access,CDP_reporting_year,Biomass_(%),Coal_(%),Gas_(%),Geothermal_(%),Hydro_(%),Nuclear_(%),Oil_(%),Solar_(%),Wind_(%),Unknown_sources_(%),City_Location
Prefeitura Municipal de Caxias do Sul,Caxias do Sul,Brazil,Latin America,2016,435664,Public,2016,0.0,25.0,0.0,0.0,72.0,0.0,0.0,0.0,3.0,0.0,"(-29.163403, -51.179668)"
Incheon Metropolitan City,Incheon,South Korea,East Asia,2015,2983484,Public,2016,0.64,0.5,15.1,0.01,0.02,0.0,63.5,0.01,0.03,20.19,"(37.456256, 126.705206)"
Prefeitura Municipal da Estância Hidromineral de Águas de São Pedro,Estância Hidromineral de Águas de São Pedro,Brazil,Latin America,2014,3007,Public,2016,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,"(-22.600006, -47.875041)"
City of Wroclaw,Wroclaw,Poland,Europe,2014,634487,Public,2017,0.0,96.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,"(51.107885, 17.038538)"
Distrito Metropolitano de Quito,Quito,Ecuador,Latin America,2010,2239191,Public,2017,0.0,0.0,0.0,0.0,69.0,0.0,31.0,0.0,0.0,0.0,"(-0.2333, -78.5167)"
City of Philadelphia,Philadelphia,USA,North America,2014,1560297,Public,2017,1.28,35.27,20.62,0.0,1.0,39.9,0.55,0.01,0.67,0.7,"(39.952335, -75.163789)"
Abasan Al-Kabira Municipality,Abasan Al-Kabira,State of Palestine,South and West Asia,2015,30000,Public,2017,5.0,17.0,18.0,0.0,0.0,0.0,50.0,10.0,0.0,0.0,"(31.323126, 34.344025)"
"City of Davis, CA","Davis, CA",USA,North America,2010,65600,Public,2017,5.0,0.0,45.0,5.0,8.0,21.0,0.0,9.0,7.0,0.0,"(38.544907, -121.740517)"
"City of London, ON","London, ON",Canada,North America,2014,375000,Public,2017,0.4,0.0,9.7,0.0,23.0,58.0,0.0,1.9,6.4,0.6,"(42.9837, -81.2497)"
Frederikshavn Kommune,Frederikshavn Kommune,Denmark,Europe,2014,23156,Public,2017,14.0,25.96,23.8,0.02,0.02,0.0,0.0,0.2,36.0,0.0,"(57.442711, 10.521006)"


In [0]:
df = df.withColumn('total_solar',col('Solar_(%)')+coalesce(df['Unknown_sources_(%)'],lit('0')))
df.show()    # Original DF now has an additional column

+--------------------+--------------------+------------------+-------------------+---------------+----------+------+------------------+-----------+--------+-------+--------------+---------+-----------+-------+---------+--------+-------------------+--------------------+------------------+
|      City_long_name|     City_short_name|           Country|             Region|Population_year|Population|Access|CDP_reporting_year|Biomass_(%)|Coal_(%)|Gas_(%)|Geothermal_(%)|Hydro_(%)|Nuclear_(%)|Oil_(%)|Solar_(%)|Wind_(%)|Unknown_sources_(%)|       City_Location|       total_solar|
+--------------------+--------------------+------------------+-------------------+---------------+----------+------+------------------+-----------+--------+-------+--------------+---------+-----------+-------+---------+--------+-------------------+--------------------+------------------+
|Prefeitura Munici...|       Caxias do Sul|            Brazil|      Latin America|           2016|    435664|Public|              201

In [0]:
import pyspark
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import lit
from pyspark.sql.functions import *
from subprocess import call

create a session

load data

check data like cleaning

In [0]:
spark = SparkSession.builder.appName('First Session').getOrCreate()

print('Spark Version: {}'.format(spark.version))

Spark Version: 3.1.2


In [0]:
print(emp_df.dtypes)        # Gives the data types of the columns
print()
print(emp_df.columns)       # Gives List of the column names
print()
print(emp_df.shape)         # Gives a tuple with count of rows and columns

In [0]:
broadcast is an optimization. The small table is broadcast to all nodes for faster join

In [0]:
joined_df = df.join(other=broadcast(Country),how='inner', on=[df.deptno==dept_df.deptno, emp_df.sal>='10']).drop(dept_df.deptno)
joined_df.show()

In [0]:
joined_df = df.select('Country','City_long_name')

In [0]:
j_df = df.select('Country','City_short_name')

In [0]:
inner_join = joined_df.join(j_df, joined_df.Country == j_df.Country,how='left')
inner_join.display()


Country,City_long_name,Country,City_short_name
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Estância Climática de São Bento do Sapucaí
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Campinas
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,São José do Rio Preto
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Natal
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Aparecida
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Cabreúva
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Brusque
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Belém
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Niterói
Brazil,Prefeitura Municipal de Caxias do Sul,Brazil,Maceió


####Are there rows with empty values
 ####Entire columns with no data
#### How is data distributed
 ####Like “date of birth: male”, “address: -1234”.
 ####Why are the same product names written in uppercase and other times in camelCase?
 
 Missing Data
 
Outliers Contaminated

Data Inconsistent

Data Invalid

Data Duplicate

Data Data Type Issues

Structural Errors

In [0]:
from pyspark import SparkContext;
from pyspark.sql import Row,SQLContext
import sys
import itertools

In [0]:
pip install streamlit

Python interpreter will be restarted.
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13451 sha256=d5adae313abdcbaaa57c2bd0b6030eb628cf525f14c5bf2ac4d2a10052e842e4
  Stored in directory: /root/.cache/pip/wheels/b7/a5/68/fe632054a5eadd531c7a49d740c50eb6adfbeca822b4eab8d4
Successfully built blinker
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-db43ea9c-82c0-4ba3-be24-c0bfb8db2078/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.


In [0]:
from pyspark import SparkContext;
from pyspark.sql import Row,SQLContext
import sys
import itertools
path = "/FileStore/tables/titles.csv"

df=spark.read.format('csv').options(header='true',inferschema='true').load(path)
s=path.split('.');
#x=str("");
#for a in s[:len(s)-1]:
	#x+=a

#x+='-stats.csv'
descibed = df.describe()
#descibed.show()
#descibed.toPandas().to_csv(x)
#print ("saved in ",x)
cols = df.columns
count = df.distinct().count()
print ("Total records ",df.count())
#for unique columns
uni=[];
for c in cols:
	x=df.groupBy(c).count().count();
	if x == count:
		print (c,"is a unique column")

cf=False
for L in range(1,len(cols)+1):
	for comb in itertools.combinations(cols,L):
		x1=df.select(list(comb)).distinct().count();
		if count==x1:
			uni=list(comb)
			if len(uni)>1:
				print (comb,"is unique combination")
			cf=True
			break
	if cf :
		break

x=df.count()

if x==count:
	print ("No duplicates rows")
else:
	print (x-count,"rows are duplicates")
	dx=df.groupBy(cols).count();
	print ("Duplicates::")
	dx.filter(dx["count"]>1).show();

Total records  178262
('emp_no', 'from_date') is unique combination
2 rows are duplicates
Duplicates::
+------+---------------+----------+----------+-----+
|emp_no|          title| from_date|   to_date|count|
+------+---------------+----------+----------+-----+
| 10001|Senior Engineer|1986-06-26|9999-01-01|    3|
+------+---------------+----------+----------+-----+



https://towardsdatascience.com/write-clean-and-solid-scala-spark-jobs-28ac4395424a    scala